In [19]:
import cv2
import os
from ultralytics import YOLO
from tkinter import Tk, Label, Button, Text, Scrollbar, END



SAVE_DIR = "E:\\swift\\ultralytics\\results\\frames"
os.makedirs(SAVE_DIR, exist_ok=True)

model = YOLO("Swift.pt")  

In [20]:
# Class costs
class_costs = {
    "biscuit": 20,
    "chips": 10,
    "chocolate": 40,
    "juice": 30,
    "milk": 50,
    "paste": 20,
    "soap": 50,
    
}



In [33]:
# Initialize video capture
#cap = cv2.VideoCapture("http://192.168.19.156:4747/video")
cap = cv2.VideoCapture("http://192.168.252.131:4747/video")
#cap = cv2.VideoCapture("http://10.91.101.1169:4747/video")

#cap = cv2.VideoCapture(0)

print("Press 's' to save frame, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    cv2.imshow("Frame", frame)

    # Capture key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord("s"):  # Save frame if "s" is pressed
        frame_name = os.path.join(SAVE_DIR, f"frame_{len(os.listdir(SAVE_DIR)) + 1}.jpg")
        cv2.imwrite(frame_name, frame)
        
        print(f"Frame saved: {frame_name}")
    elif key == ord("q"):  # Quit if "q" is pressed
        break

cap.release()
cv2.destroyAllWindows()


# Process saved images

saved_images = [os.path.join(SAVE_DIR, img) for img in os.listdir(SAVE_DIR) if img.endswith(".jpg")]

if not saved_images:
    print("No frames saved to process.")
else:
    print("Processing saved frames...")
    results = model(saved_images)  # Run inference on saved images

    # Initialize a variable to store detected class names and costs
    detected_classes_with_costs = []

    # Process results
    for result in results:
        if result.boxes:
            class_names_and_costs = [
                {
                    "class_name": model.names[int(box.cls)],
                    "cost": class_costs.get(model.names[int(box.cls)], "N/A"),  # Default to "N/A" if no cost is specified
                }
                for box in result.boxes
            ]
        else:
            class_names_and_costs = []

        detected_classes_with_costs.append(class_names_and_costs)
        result.show()  # Display result
        result.save()  # Save visualized result

    print("********************************")
    total_cost = 0
    bill_details = []
    for image_idx, detections in enumerate(detected_classes_with_costs):
        bill_details.append(f"Image {image_idx + 1}:\n")
        for detection in detections:
            bill_details.append(f"Class: {detection['class_name']}, Cost: {detection['cost']}\n")
            if isinstance(detection["cost"], int):  # Ensure cost is numeric
                total_cost += detection["cost"]

    bill_details.append("********************************\n")
    bill_details.append(f"Total Cost: {total_cost}\n")
    bill_details.append("********************************\n")

    # Function to display the bill in a full-screen popup
    def show_bill_popup():
        root = Tk()
        root.title("Bill Details")
        root.state('zoomed')  # Make the window full-screen

        # Title Label
        Label(root, text="Bill Details", font=("Helvetica", 16, "bold")).pack(pady=10)

        # Text widget with scrollbar
        text_widget = Text(root, wrap="word", font=("Helvetica", 16))
        scrollbar = Scrollbar(root, command=text_widget.yview)
        text_widget.configure(yscrollcommand=scrollbar.set)
        text_widget.pack(side="left", fill="both", expand=True, padx=20, pady=20)
        scrollbar.pack(side="right", fill="y")

        # Insert the bill details
        text_widget.insert(END, "".join(bill_details))
        text_widget.config(state="disabled") 

        # Close button
        Button(root, text="Close", command=root.destroy, font=("Helvetica", 16), bg="red", fg="white").pack(pady=10)

        root.mainloop()

    # Show the popup
    show_bill_popup()

    print("********************************")
    print(f"Total Cost: {total_cost}")
    print("********************************")

Press 's' to save frame, 'q' to quit.
No frames saved to process.
